# Importações

In [149]:
import pandas as pd
import numpy as np

# Exploratória

In [150]:
df_raw = pd.read_csv('./data/raw/raw_data.csv')
df_raw.head(5)

,id,created_at,updated_at,deleted_at,name_hash,email_hash,address_hash,birth_date,status,version,city,state,neighborhood,last_date_purchase,average_ticket,items_quantity,all_revenue,all_orders,recency,marketing_source
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,08/15/17 07:05 AM,01/14/21 11:23 AM,NaN,312d206168a318614897e8ccac43bff9,83eb3aed9a44377df80ce876dce92c9a,8b4bfaa0cbc41a16f46da15ddcd6a907,07/10/74 12:00 AM,active,2.31.7,Peixoto da Praia,AM,Aparecida 7ª Seção,01/14/21 11:23 AM,151.142942,10,906.857651,6,35,crm
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,12/31/19 09:53 PM,01/08/21 11:23 AM,NaN,de448fcb47d0d6a873b2eef52b5ee595,72678bb35e2ac84ed373e81dd9dca28c,22f1cfa1847f38da3f3cb114dd2b9247,07/06/40 12:00 AM,paused,3.30.12,Fernandes,RR,Santa Isabel,01/08/21 11:23 AM,236.991790,4,236.991790,1,41,organic_search
2,dc067cd2-c021-42bd-8c0e-beb267280e66,03/07/19 11:46 PM,01/07/21 11:23 AM,NaN,cb09e447ddc38283373d56bb46498e6a,668f4ee9add29c7bd02c485f1b7509e3,6cb47446a086ee6483b3eb954f11467a,03/18/63 12:00 AM,active,3.28.9,Lopes,RR,Estrela,01/07/21 11:23 AM,211.955597,13,2331.511572,11,42,organic_search
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,07/21/18 10:17 AM,01/10/21 11:23 AM,NaN,52593437a405b11b3557170680ef80c8,d3fb45188d95c8d7cc49da5b4f727c86,0a6f0c54db1e6f19347f96b50f8092a4,11/21/80 12:00 AM,active,3.34.3,Campos do Campo,PE,Confisco,01/10/21 11:23 AM,204.113227,8,1224.679359,6,39,organic_search
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,06/08/18 12:09 PM,01/18/21 11:23 AM,NaN,dbda4b778a966c21904238ed2d2005db,a0f76bc49b4c43327b536da6e1a1465e,143b9f169b4fa1692f6d79b5682169b5,07/07/59 12:00 AM,active,3.19.8,das Neves,RJ,Vila Suzana Segunda Seção,01/18/21 11:23 AM,252.940997,9,2023.527980,8,31,crm


In [151]:
df_raw.isna().sum()

id                       0
created_at               0
updated_at               0
deleted_at            9495
name_hash                0
email_hash               0
address_hash             0
birth_date               0
status                   0
version                  0
city                     0
state                    0
neighborhood             0
last_date_purchase       0
average_ticket           0
items_quantity           0
all_revenue              0
all_orders               0
recency                  0
marketing_source         0
dtype: int64

In [152]:
df_raw.dtypes

id                     object
created_at             object
updated_at             object
deleted_at             object
name_hash              object
email_hash             object
address_hash           object
birth_date             object
status                 object
version                object
city                   object
state                  object
neighborhood           object
last_date_purchase     object
average_ticket        float64
items_quantity          int64
all_revenue           float64
all_orders              int64
recency                 int64
marketing_source       object
dtype: object

# Transformação

## Funções

In [153]:
def obj_to_datetime(col):
     return pd.to_datetime(col, format='%m/%d/%y %H:%M %p')

def diferenca_datas(series_final, series_inicial, periodo):
     return ((series_final - series_inicial) / np.timedelta64(1, periodo))


Necessário haver tratamento com relação ao ano. Uma vez que, de acordo com os padrões POSIX e ISO C, valores menores do que 69 serão mapeados como sendo do século XXI.

*Fonte*: https://docs.python.org/3/library/time.html

In [154]:
def corrige_idade(array1):
    return array1.where(array1 >= 0, 100 + array1)

## Tabela por usuário

In [155]:
df_usuario = pd.DataFrame()

In [156]:
df_usuario['id'] = df_raw['id']

### Cálculo de tempo de clientela - Meses

In [157]:
aux = pd.DataFrame()
aux['criacao'] = obj_to_datetime(df_raw['created_at'])
aux['exclusao'] = obj_to_datetime(df_raw['deleted_at'])
aux['duracao'] = diferenca_datas(aux.exclusao, aux.criacao, 'M')

In [158]:
aux['duracao'] = aux['duracao'].fillna(-1)
aux['duracao'] = aux['duracao'].astype(int)

In [159]:
# Conforme colunas 'recency' e 'last_update', o arquivo data de 17 de fevereiro de 2021

aux['ultima_aquisicao'] = obj_to_datetime(df_raw['last_date_purchase'])
aux['ultima_aquisicao'] = obj_to_datetime(df_raw['last_date_purchase'])
aux['intervalo_de_aquisicao'] = ((np.where(aux['exclusao'].notna(), aux['exclusao'], np.datetime64("2021-02-17")) - aux.ultima_aquisicao) / np.timedelta64(1, 'M'))

In [161]:
df_usuario[['meses_assinatura', 'intervalo_de_inatividade']] = aux[['duracao', 'intervalo_de_aquisicao']]
df_usuario

,id,meses_assinatura,intervalo_de_inatividade
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,-1,1.081474
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,-1,1.275022
2,dc067cd2-c021-42bd-8c0e-beb267280e66,-1,1.307280
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,-1,1.210506
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,-1,0.952442
...,...,...,...
9995,90c9fd5f-80d2-4f59-9970-6b3e6dac5f57,-1,0.533087
9996,0edb7aec-8816-4904-bcd7-cbe141391441,-1,0.952442
9997,30130652-d616-4b40-adbb-189e3351016c,-1,1.436313
9998,37b52952-afdd-425a-9149-4012ea48ac87,-1,1.242764


In [162]:
del aux

### Cálculo de Idade

In [163]:
aux = pd.DataFrame()

aux['nascimento'] = obj_to_datetime(df_raw['birth_date'])
aux['idade'] = diferenca_datas(np.datetime64('today', 'D'), aux.nascimento, 'Y')
aux['idade'] = corrige_idade(aux['idade']).astype(int)

In [164]:
df_usuario['idade'] = aux.idade

In [165]:
del aux

### Binarização de status

In [166]:
df_usuario['ativo'] = np.where(df_raw.status == 'canceled', 0, 1)
df_usuario

,id,meses_assinatura,intervalo_de_inatividade,idade,ativo
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,-1,1.081474,48,1
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,-1,1.275022,82,1
2,dc067cd2-c021-42bd-8c0e-beb267280e66,-1,1.307280,60,1
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,-1,1.210506,42,1
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,-1,0.952442,63,1
...,...,...,...,...,...
9995,90c9fd5f-80d2-4f59-9970-6b3e6dac5f57,-1,0.533087,61,1
9996,0edb7aec-8816-4904-bcd7-cbe141391441,-1,0.952442,53,1
9997,30130652-d616-4b40-adbb-189e3351016c,-1,1.436313,63,1
9998,37b52952-afdd-425a-9149-4012ea48ac87,-1,1.242764,65,1


### Demais

In [167]:
df_usuario[['versao', 'cidade', 'estado', 'gasto_medio', 'media_de_itens', 'gasto_total', 'total_de_pedidos', 'canal_marketing']] = df_raw[['version', 'city', 'state', 'average_ticket', 'items_quantity', 'all_revenue', 'all_orders', 'marketing_source']]
df_usuario

,id,meses_assinatura,intervalo_de_inatividade,idade,ativo,versao,cidade,estado,gasto_medio,media_de_itens,gasto_total,total_de_pedidos,canal_marketing
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,-1,1.081474,48,1,2.31.7,Peixoto da Praia,AM,151.142942,10,906.857651,6,crm
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,-1,1.275022,82,1,3.30.12,Fernandes,RR,236.991790,4,236.991790,1,organic_search
2,dc067cd2-c021-42bd-8c0e-beb267280e66,-1,1.307280,60,1,3.28.9,Lopes,RR,211.955597,13,2331.511572,11,organic_search
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,-1,1.210506,42,1,3.34.3,Campos do Campo,PE,204.113227,8,1224.679359,6,organic_search
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,-1,0.952442,63,1,3.19.8,das Neves,RJ,252.940997,9,2023.527980,8,crm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,90c9fd5f-80d2-4f59-9970-6b3e6dac5f57,-1,0.533087,61,1,4.30.12,Souza,RN,215.079785,5,1075.398927,5,paid_search
9996,0edb7aec-8816-4904-bcd7-cbe141391441,-1,0.952442,53,1,4.28.13,Silveira,AL,254.525826,10,2290.732434,9,organic_search
9997,30130652-d616-4b40-adbb-189e3351016c,-1,1.436313,63,1,3.18.11,Martins,SC,208.741902,8,1252.451411,6,paid_search
9998,37b52952-afdd-425a-9149-4012ea48ac87,-1,1.242764,65,1,5.51.6,Gomes das Flores,PA,211.174796,9,0.000000,0,direct
